In [1]:
from bs4 import BeautifulSoup as soup
import re
import time
from selenium import webdriver
import json

In [2]:
def get_products_from_url(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    driver = webdriver.Chrome("driver/chromedriver", chrome_options=options)
    driver.get(url)
    time.sleep(5)
    page_source = driver.page_source
    text = soup(page_source, "html.parser")
    
    whole_links = []
    data = text.findAll('div',attrs={'class':"prodbox"})
    for div in data:
        links = div.findAll('a')
        for a in links:
            if ".htm" in a['href']:
                whole_links.append("https://online.asbis.sk/"+a['href'])
    return whole_links

In [3]:
def get_products_from_urls(url):
    urls = []
    for i in range(1,20):
        urls.append(url+"#cls=spresenttrees&strid="+ re.search(r'\d+', url).group()+"&page="+str(i))
    
    all_links = []
    
    for u in urls:
        new_links = get_products_from_url(u)
        if len(new_links) == 0:
            break
        all_links.append(new_links)
        
    links = list(set([item for sublist in all_links for item in sublist]))
    return links
    


In [4]:
def get_product_info_from_url(url,driver):
    driver.get(url)
    time.sleep(10)
    page_source = driver.page_source
    text = soup(page_source, "html.parser")
    prod_name = text.find('h1').getText()
    try:
        prod_description = re.sub(r"(\w)([A-Z])", r"\1 \2", text.find('div',attrs={'class':"wrap"}).getText()).split("Popis produktu:",1)[1]
    except:
        prod_description = re.sub(r"(\w)([A-Z])", r"\1 \2", text.find('div',attrs={'class':"wrap"}).getText())
    prod_categories = re.sub(r"(\w)([A-Z])", r"\1 \2", text.find('ul',attrs={'class':"stoitem_navi"}).getText()).replace("\n", "").replace("\xa0", "").replace("Hlavná strana", "").split(">") 
    prod_categories.remove("")
    return prod_name, prod_categories, prod_description

In [5]:
def get_products_and_info_from_url(url):
    print("Getting Links")
    links = get_products_from_urls(url)
    print("Links Retrieved")
    link_dicts = []
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    driver = webdriver.Chrome("driver/chromedriver", chrome_options=options)
    for link in links:
        prod_name, prod_categories, prod_description = get_product_info_from_url(link,driver)
        link_dict = {"prod_name": prod_name,
                     "prod_categories": prod_categories,
                     "prod_description": prod_description,
                    }
        link_dicts.append(link_dict)
    return link_dicts

In [6]:
link_dicts = get_products_and_info_from_url("https://online.asbis.sk/tablety-smartfony-telefony_c11914.html")

Getting Links


/Users/adambako/PHALANX/webscraper/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


Links Retrieved


/Users/adambako/PHALANX/webscraper/venv/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


WebDriverException: Message: unknown error: net::ERR_CONNECTION_REFUSED
  (Session info: headless chrome=87.0.4280.67)


In [ ]:
category = {"products": prods}

with open('product_data_asbis_phones.json', 'w') as fp:
    json.dump(category, fp)